In [25]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from google.colab import userdata

In [5]:
students_df = pd.read_csv("train.csv")

In [6]:
id_estudiantes = students_df["ID"].copy()

In [7]:
students_df = students_df.drop(['ID'], axis=1)
students_df = students_df.fillna('Miss')

In [8]:
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

def replace_column_with_onehot(d, col):
    assert sum(d[col].isna())==0, "column must have no NaN values"
    values = np.unique(d[col])
    k = to_onehot(d[col].values)
    r = d.join(pd.DataFrame(k, columns=["%s_%s"%(col, values[i]) for i in range(k.shape[1])], index=d.index), how='outer')
    del(r[col])
    return r

columns_to_onehot =[
    'F_TIENEINTERNET',
    'E_PAGOMATRICULAPROPIO',
]
onehoted_df = students_df.copy()
for col in columns_to_onehot:
  onehoted_df = replace_column_with_onehot(onehoted_df, col)
onehoted_df

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_Miss,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Técnica o tecnológica incompleta,Si,Si,N,...,0.322,0.208,0.310,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,Técnica o tecnológica completa,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Secundaria (Bachillerato) completa,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,No sabe,Si,No,N,...,0.485,0.172,0.252,0.190,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Primaria completa,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Secundaria (Bachillerato) completa,Si,No,N,...,0.237,0.271,0.271,0.311,0,0,1,0,0,1
692496,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Primaria incompleta,Si,No,N,...,0.314,0.240,0.278,0.260,0,0,1,0,1,0
692497,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Secundaria (Bachillerato) completa,Si,No,N,...,0.286,0.240,0.314,0.287,0,0,1,0,0,1
692498,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,Primaria completa,No,No,N,...,0.132,0.426,0.261,0.328,0,1,0,0,0,1


In [9]:
def replace_category_for_numbers(d, col, vals_to_change):
  r = d.copy()
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())
  return r

In [10]:
data_education_dict = {
    'Miss': -2,
    'No Aplica': -1,
    'Ninguno': 0,
    'No sabe': 1,
    'Primaria incompleta': 2,
    'Primaria completa': 3,
    'Secundaria (Bachillerato) incompleta': 4,
    'Secundaria (Bachillerato) completa': 5,
    'Técnica o tecnológica incompleta': 6,
    'Técnica o tecnológica completa': 7,
    'Educación profesional incompleta': 8,
    'Educación profesional completa': 9,
    'Postgrado': 10,
}

data_to_convert = {
    'E_HORASSEMANATRABAJA': {
        'Miss': 0,
        '0': 1,
        'Menos de 10 horas': 2,
        'Entre 11 y 20 horas': 3,
        'Entre 21 y 30 horas': 4,
        'Más de 30 horas': 5,
    },
    'F_ESTRATOVIVIENDA': {
        'Miss': -1,
        'Sin Estrato': 0,
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6,
    },
    'RENDIMIENTO_GLOBAL': {
        'bajo': 0,
        'medio-bajo': 1,
        'medio-alto': 2,
        'alto': 3,
    },
    'F_EDUCACIONPADRE':data_education_dict,
    'F_EDUCACIONMADRE':data_education_dict,
}

for col, transform_dict in data_to_convert.items():
    onehoted_df = replace_category_for_numbers(
        onehoted_df,
        col,
        transform_dict)

onehoted_df.head()

/tmp/ipython-input-3900146150.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_Miss,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,2,3,6,Si,Si,N,...,0.322,0.208,0.310,0.267,0,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,1,3,7,Si,No,N,...,0.311,0.215,0.292,0.264,0,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,5,3,5,Si,No,N,...,0.297,0.214,0.305,0.264,0,0,1,0,1,0
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,1,4,1,Si,No,N,...,0.485,0.172,0.252,0.190,0,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,4,3,3,Si,Si,N,...,0.316,0.232,0.285,0.294,0,0,1,0,1,0


In [11]:
def replace_word(word, sentence: str):
  return sentence.replace(" DE ", " ")

def replace_wrong_words(sentence):
  result = sentence
  wrong_words_dict = {
      '3DEG ': '',
      ' PRIMARIA ': ' ',
      ' PRIMARIA:': ' ',
      'INGENIER?A': 'INGENIERIA',
      'II': 'I',
      'EDUCACI?N': 'EDUCACION',
      'F?SICA': 'FISICA',
      'RECREACI?N': 'RECREACION',
      '?NFASIS': 'ENFASIS',
      'INGL?S': 'INGLES',
      'DISE?O': 'DISENO',
      'COMUNICACI?N': 'COMUNICACION',
      'QU?MICA': 'QUIMICA',
      'MATEM?TICAS': 'MATEMATICAS',
      'EL?CTRICA': 'ELECTRICA',
      'M?SICA': 'MUSICA',
      'DISE??O': 'DISENO',
      'COMUNICACI??N': 'COMUNICACION',
      'QU?MICA': 'QUIMICA',
      'GESTI?N': 'GESTION',
      'GR??FICA': 'GRAFICA',
      'B?SICA': 'BASICA',
      'ADMINISTRACI?N': 'ADMINISTRACION',
      'P?BLICA': 'PUBLICA',
      'FARMAC?UTICA': 'FARMACEUTICA',
      'FILOSOF?A': 'FILOSOFIA',
      'ESC?NICAS': 'ESCENICAS',
      'ECONOM?A': 'ECONOMIA',
      'GASTRONOM?A': 'GASTRONOMIA',
      'GEOLOG?A': 'GEOLOGIA',
      'LOG?STICA': 'LOGISTICA',
      'ART?STICA': 'ARTISTICA',
      'PEDAGOG?A': 'PEDAGOGIA',
      ' DE ': ' ',
      ' EN ': ' ',
      '  ': ' ',
      '.': '',
      '- ': ' ',
  }
  for wrong, well in wrong_words_dict.items():
    if wrong in result: result = result.replace(wrong, well)
  return result

In [12]:
onehoted_df = onehoted_df.drop(['F_TIENEINTERNET_Miss'], axis=1)
print (onehoted_df.shape)
onehoted_df

(692500, 23)


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,E_PAGOMATRICULAPROPIO_No,E_PAGOMATRICULAPROPIO_Si
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,2,3,6,Si,Si,N,...,2,0.322,0.208,0.310,0.267,0,1,0,1,0
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,1,3,7,Si,No,N,...,0,0.311,0.215,0.292,0.264,1,0,0,1,0
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,5,3,5,Si,No,N,...,0,0.297,0.214,0.305,0.264,0,1,0,1,0
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,1,4,1,Si,No,N,...,3,0.485,0.172,0.252,0.190,0,1,0,1,0
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,4,3,3,Si,Si,N,...,1,0.316,0.232,0.285,0.294,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,3,2,5,Si,No,N,...,2,0.237,0.271,0.271,0.311,0,1,0,0,1
692496,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,5,3,2,Si,No,N,...,0,0.314,0.240,0.278,0.260,0,1,0,1,0
692497,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,2,3,5,Si,No,N,...,1,0.286,0.240,0.314,0.287,0,1,0,0,1
692498,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,2,1,3,No,No,N,...,0,0.132,0.426,0.261,0.328,1,0,0,0,1


In [13]:
final_student_df = replace_column_with_onehot(onehoted_df, 'E_PRGM_DEPARTAMENTO')
y = final_student_df["RENDIMIENTO_GLOBAL"].values
X = final_student_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)
X.E_PRGM_ACADEMICO = X.E_PRGM_ACADEMICO.astype("category")
X.E_VALORMATRICULAUNIVERSIDAD = X.E_VALORMATRICULAUNIVERSIDAD.astype("category")
X.PERIODO_ACADEMICO = X.PERIODO_ACADEMICO.astype("category")
del final_student_df
print (X.shape, y.shape)

(692500, 52) (692500,)


In [14]:
id_estudiantes = id_estudiantes.loc[X.index]

In [15]:
def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train, eval_set=[(X_test, y_test)])

    train_score = estimator.score(X_train, y_train)
    test_score = estimator.score(X_test, y_test)

    return estimator, train_score, test_score

In [16]:
# Convertir columnas tipo object a category
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')

In [17]:
# Ensure y is a pandas Series for easier manipulation and value counts.
# The original y might be a numpy array or a Series. Converting it explicitly.
y_original_series = pd.Series(y)

# Convert 'Miss' to -1 and then to integer.
y_processed = y_original_series.replace('Miss', -1).astype(int)

# Identify classes with only one sample
class_counts = y_processed.value_counts()
problematic_classes = class_counts[class_counts == 1].index.tolist()

if problematic_classes:
    print(f"Warning: The following classes have only one member and cannot be used for stratified sampling: {problematic_classes}")
    print("These samples will be removed from the dataset before splitting.")

    # Get the indices of samples to keep (i.e., those not belonging to problematic classes)
    indices_to_keep = ~y_processed.isin(problematic_classes)

    # Filter X and y_processed
    X_filtered = X.loc[indices_to_keep]
    y_filtered = y_processed.loc[indices_to_keep].values # Convert Series to numpy array for train_test_split

    print(f"Original X shape: {X.shape}, Filtered X shape: {X_filtered.shape}")
    print(f"Original y shape: {y_processed.shape}, Filtered y shape: {y_filtered.shape}")

    # Use the filtered data for train_test_split
    Xtr, Xts, ytr, yts = train_test_split(
        X_filtered, y_filtered,
        test_size=0.2,
        random_state=42,
        stratify=y_filtered
    )
else:
    print("No problematic classes with only one member found after conversion.")
    # If no problematic classes, just use the converted y for splitting
    Xtr, Xts, ytr, yts = train_test_split(
        X, y_processed.values, # Use the converted y as numpy array
        test_size=0.2,
        random_state=42,
        stratify=y_processed.values
    )

print("Datos divididos:")
print(f"- Entrenamiento: X={Xtr.shape}, y={ytr.shape}")
print(f"- Prueba: X={Xts.shape}, y={yts.shape}")

No problematic classes with only one member found after conversion.
Datos divididos:
- Entrenamiento: X=(554000, 52), y=(554000,)
- Prueba: X=(138500, 52), y=(138500,)


In [26]:
from sklearn.tree import DecisionTreeClassifier # Import DecisionTreeClassifier
import numpy as np

estimator_dt = DecisionTreeClassifier( # Changed name to estimator_dt
    max_depth=10, # Removed n_estimators and n_jobs
    random_state=42
)

print("Iniciando entrenamiento...")

# Ensure Xtr and Xts are fully numerical before fitting

# 1. Handle INDICADOR columns: Replace 'Miss' with NaN and convert to float
indicator_cols = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
for col in indicator_cols:
    if col in Xtr.columns:
        # Use .copy() to avoid SettingWithCopyWarning
        Xtr[col] = pd.to_numeric(Xtr[col].replace('Miss', np.nan), errors='coerce')
        Xts[col] = pd.to_numeric(Xts[col].replace('Miss', np.nan), errors='coerce')

# 2. Convert all remaining categorical columns to one-hot encoding
# pd.get_dummies will handle 'category' and 'object' dtypes
Xtr_processed = pd.get_dummies(Xtr)
Xts_processed = pd.get_dummies(Xts)

# 3. Align columns - crucial for consistent feature sets between train and test
# This step ensures both train and test sets have the same columns in the same order
# and handles categories present in one but not in the other.

# Get all unique columns from both processed DataFrames
all_cols = Xtr_processed.columns.union(Xts_processed.columns)

# Reindex both DataFrames to have all_cols, filling new columns with 0
Xtr_processed = Xtr_processed.reindex(columns=all_cols, fill_value=0)
Xts_processed = Xts_processed.reindex(columns=all_cols, fill_value=0)

# 4. Fill any NaNs that might have been introduced (e.g., from INDICADOR columns)
# For Decision Tree, mean imputation is a common strategy.
# Let's fill with the mean for simplicity, using train's mean for both train and test.
for col in Xtr_processed.columns:
    if Xtr_processed[col].isnull().any():
        mean_val = Xtr_processed[col].mean()
        Xtr_processed[col] = Xtr_processed[col].fillna(mean_val)
        Xts_processed[col] = Xts_processed[col].fillna(mean_val)


# Now fit the estimator with the fully numerical and aligned data
estimator_dt.fit(Xtr_processed, ytr) # Changed estimator_rf to estimator_dt

print("Entrenamiento finalizado.")

Iniciando entrenamiento...
Entrenamiento finalizado.


In [29]:
y_pred_dt = estimator_dt.predict(Xts_processed) # Changed to estimator_dt and y_pred_dt

# Scores de precisión (Accuracy)
train_score_dt = estimator_dt.score(Xtr_processed, ytr) # Changed to estimator_dt and train_score_dt
test_score_dt = estimator_dt.score(Xts_processed, yts) # Changed to estimator_dt and test_score_dt

print(f"Accuracy de entrenamiento (In-Sample): {train_score_dt:.4f}") # Changed to train_score_dt
print(f"Accuracy de prueba (Out-of-Sample): {test_score_dt:.4f}") # Changed to test_score_dt

Accuracy de entrenamiento (In-Sample): 0.3951
Accuracy de prueba (Out-of-Sample): 0.3882


In [30]:
# Mapeo de etiquetas (si las clases son 0, 1, 2, 3)
mapeo = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}

# Reporte de Clasificación (usa la etiqueta numérica real y mapeo para los nombres)
print("\n--- Informe de Clasificación en el conjunto de prueba ---")
print(classification_report(yts, y_pred_dt, target_names=mapeo.values())) # Changed y_pred_rf to y_pred_dt

# Resultados en formato de etiquetas (opcional, para visualización)
yts_etiquetas = pd.Series(yts).map(mapeo).reset_index(drop=True)
y_pred_etiquetas_dt = pd.Series(y_pred_dt).map(mapeo) # Changed y_pred_etiquetas_rf to y_pred_etiquetas_dt

resultados = pd.DataFrame({
    'y_real': yts_etiquetas,
    'y_predicha': y_pred_etiquetas_dt # Changed y_pred_etiquetas_rf to y_pred_etiquetas_dt
})
print("\nPrimeras 5 predicciones vs valores reales:")
print(resultados.head())


--- Informe de Clasificación en el conjunto de prueba ---
              precision    recall  f1-score   support

        bajo       0.40      0.52      0.45     34597
  medio-bajo       0.31      0.24      0.27     34455
  medio-alto       0.30      0.25      0.27     34324
        alto       0.51      0.54      0.52     35124

    accuracy                           0.39    138500
   macro avg       0.38      0.39      0.38    138500
weighted avg       0.38      0.39      0.38    138500


Primeras 5 predicciones vs valores reales:
       y_real  y_predicha
0        bajo  medio-alto
1  medio-bajo  medio-bajo
2        alto  medio-alto
3        alto        alto
4  medio-bajo  medio-alto
